# console capture and replay for notebooks

This notebook shows an example of an small utility that can help capture the output sent to stdout by a long-running process in a notebook.

In [1]:
from __future__ import print_function

from dl_utils import console_capture as cc

A preliminary check: this cell should print

  `789...`

since the carriage return should make it overwrite all previous output. If not, this is possibly a [buggy notebook version](https://github.com/jupyter/notebook/issues/1970). Update notebook to at least 4.3.1


In [2]:
a = '123...\r456...\r789...'
print(a)

789...


Backspaces are not correctly processed by notebooks, though (they are ignored). So this one will not print what it should:

In [3]:
a = '123...\r456...\r789\b\b\babc...'
print(a)

789abc...


# High-level API

## Process example

To use the high-level API we need out processing task in the form of a [callable](https://docs.python.org/3/library/functions.html#callable)  (a function, method, or in general any object with a `__call__` [attribute](https://docs.python.org/3/reference/datamodel.html#object.__call__)), with arbitrary arguments

In [4]:
import time

def processing_call( num, msg, **kwargs ):
    '''
    A function that basically does nothing, but takes its time to do 
    that nothing and prints out progress reports while it is doing it
    '''
    start = time.time()
    # Outer loop: 'num' times
    for j in range(num):
        # Inner loop: 10 times
        for i in range(10):
            # Print progress reports. Use control characters to overwrite the line
            print( "\b\b\b\b\b\b\b\b\b\b\b\r", '{:2} {:2} {:5.2f}'.format( j+1, i+1, time.time()-start ), end=' ' )
            time.sleep(0.10)
        print('<iteration done>') # next line
        time.sleep(1)
    print( "Done! Msg={} Param={}".format(msg,kwargs) )

## Basic style
The basic procedure is creating a wrap object by passing it the callable with its required arguments ...

In [5]:
p = cc.ProcessWrap( processing_call, 4, "Hi", param="I'm a process" )

... and then calling the `process()` method on that object. It appears to be the same as executing the callable directly: the process is carried out, the cell appears as **`[*]`** (i.e. working) and the notebook as _busy_ and the progress results printed by the callable are written back to the notebook.

In [6]:
p.process()

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.95 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


But behind the scenes, the printed data has been written to a temporal file, whose contents are automatically loaded upon termination as internal data object (and the file deleted), so they can be reprinted at will:

In [7]:
p.show()

----- DONE -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.95 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


### accessing raw data

The same output is also available in raw form through the `data` attribute of the object:

In [8]:
p.data

'\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  1  0.00 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  2  0.10 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  3  0.20 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  4  0.30 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  5  0.40 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  6  0.50 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  7  0.61 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  8  0.71 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  9  0.81 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1 10  0.91 <iteration done>\n\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  1  2.01 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  2  2.11 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  3  2.21 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  4  2.31 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  5  2.42 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  6  2.52 \x08\x08\x08\x08\x08\x08\x08\x08\x08\

Obviously all those control characters make the data a bit difficult to read. We can use the provided `clean_string()` function, which applies backspaces (`\b`) and carriage returns (`\r`) as they would be processed were the data sent to standard output:

In [9]:
cc.clean_string( p.data )

'  1 10  0.91 <iteration done>\n  2 10  2.92 <iteration done>\n  3 10  4.94 <iteration done>\n  4 10  6.95 <iteration done>\nDone! Msg=Hi Param={\'param\': "I\'m a process"}\n'

### closing and reopening the notebook window

More importantly, the `process()` call, though it seems to be blocking it is doing so by actually looping over a check-sleep cycle. The actual process is being carried out in a **background thread**, which will continue saving its results to the temporal file even if the browser window is closed. 

Opening the notebook later on and executing the `show()` method will print out the results stored in the file. Moreover, it will reconnect the standard output of the process with the notebook cell, so that further output gets back to the notebook. We can test it with a longer running process:

In [10]:
p2 = cc.ProcessWrap( processing_call, 50, "Hi" )

In [ ]:
p2.process()  
# after launching execution of this cell, save the notebook & close the browser window 
# (confirm the warning about leaving the page)

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.94 <iteration done>
  4 10  6.95 <iteration done>
  5 10  8.96 <iteration done>
  6 10 10.98 <iteration done>
  7 10 12.99 <iteration done>
  8 10 15.00 <iteration done>
  9 10 17.01 <iteration done>
 10 10 19.03 <iteration done>
 11 10 21.04 <iteration done>
 12 10 23.06 

In [12]:
# Then come back again here, INTERRUPT the kernel and execute this cell. 
# It should print all output received so far, and continue from there
p2.show() 

----- RUNNING -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.94 <iteration done>
  4 10  6.95 <iteration done>
  5 10  8.96 <iteration done>
  6 10 10.98 <iteration done>
  7 10 12.99 <iteration done>
  8 10 15.00 <iteration done>
  9 10 17.01 <iteration done>
 10 10 19.03 <iteration done>
 11 10 21.04 <iteration done>
 12 10 23.06 <iteration done>
 13 10 25.07 <iteration done>
 14 10 27.08 <iteration done>
 15 10 29.10 <iteration done>
 16 10 31.11 <iteration done>
 17 10 33.12 <iteration done>
 18 10 35.13 <iteration done>
 19 10 37.14 <iteration done>
 20 10 39.14 <iteration done>
 21 10 41.15 <iteration done>
 22 10 43.16 <iteration done>
 23 10 45.17 <iteration done>
 24 10 47.18 <iteration done>
 25 10 49.19 <iteration done>
 26 10 51.21 <iteration done>
 27 10 53.22 <iteration done>
 28 10 55.23 <iteration done>
 29 10 57.24 <

There is one caveat: as seen in the previous cell, the `process()` call was blocking the notebook (by doing the mentioned check-sleep cycle), so coming back to the notebook (opening its window again) will make it appear unresponsive. 

You will need to _interrupt_ the kernel to stop that `process()` cell so that it is possible to execute a `show()` cell. This is not problematic, since what it is being interrupted is the foreground process, not the background thread that it is doing the real processing. But nevertheless if it is known for sure that the window will be closed and reopened, it will be better to use the non-blocking call below.

## Non-blocking usage

This modality works the same way, but adding the `block=False` parameter to the `process()` method: 

In [8]:
pn = cc.ProcessWrap( processing_call, 50, "Look! NonBlocking!" )

In [9]:
pn.process( block=False )
# Now the notebook appears free to use (not busy), though the background thread 
# is still pushing text to the cell output
# We can save & close the notebook here. There will probable still be a "confirm" warning.

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.94 <iteration done>
  4 10  6.95 <iteration done>
  5 10  8.97 <iteration done>
  6 10 10.98 <iteration done>
  7 10 12.99 <iteration done>


In [10]:
# After coming back, in this modality it is no longer necessary to interrupt the kernel. 
# Just execute this cell to reconnect with the processing cell output
pn.show()

----- RUNNING -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.94 <iteration done>
  4 10  6.95 <iteration done>
  5 10  8.97 <iteration done>
  6 10 10.98 <iteration done>
  7 10 12.99 <iteration done>
  8 10 15.00 <iteration done>
  9 10 17.01 <iteration done>
 10 10 19.02 <iteration done>
 11 10 21.03 <iteration done>
 12 10 23.04 <iteration done>
 13 10 25.05 <iteration done>
 14 10 27.06 <iteration done>
 15 10 29.08 <iteration done>
 16 10 31.09 <iteration done>
 17 10 33.10 <iteration done>
 18 10 35.11 <iteration done>
 19 10 37.13 <iteration done>
 20 10 39.14 <iteration done>
 21 10 41.16 <iteration done>
 22 10 43.17 <iteration done>
 23 10 45.19 

In [11]:
# We can still print the collected output, after it has finished
pn.show()

----- DONE -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.94 <iteration done>
  4 10  6.95 <iteration done>
  5 10  8.97 <iteration done>
  6 10 10.98 <iteration done>
  7 10 12.99 <iteration done>
  8 10 15.00 <iteration done>
  9 10 17.01 <iteration done>
 10 10 19.02 <iteration done>
 11 10 21.03 <iteration done>
 12 10 23.04 <iteration done>
 13 10 25.05 <iteration done>
 14 10 27.06 <iteration done>
 15 10 29.08 <iteration done>
 16 10 31.09 <iteration done>
 17 10 33.10 <iteration done>
 18 10 35.11 <iteration done>
 19 10 37.13 <iteration done>
 20 10 39.14 <iteration done>
 21 10 41.16 <iteration done>
 22 10 43.17 <iteration done>
 23 10 45.19 <iteration done>
 24 10 47.20 <iteration done>
 25 10 49.22 <iteration done>
 26 10 51.23 <iteration done>
 27 10 53.25 <iteration done>
 28 10 55.26 <iteration done>
 29 10 57.28 <iteration done>
 30 10 59.29 <iteration done>
 31 10 61.31 <iteration done>
 32 10 63.32 <iteration done>
 33 10 65.34 <iteration

## Additional options

### verbose

Adding `verbose=False` to the `process()` method will suppress output (but still save to the temporal file)

In [14]:
pq = cc.ProcessWrap( processing_call, 3, "Now Quiet!" )
pq.process( verbose=False )
# This will take a few seconds to run, with no output produced

In [15]:
# After finishing, we can now print the captured data
pq.show()

----- DONE -----
  1 10  0.90 <iteration done>
  2 10  2.91 <iteration done>
  3 10  4.91 <iteration done>
Done! Msg=Now Quiet! Param={}


### delete

The temporal file is automatically deleted upon task termination. This is usually not a problem because the data in the file is loaded onto the `ProcessWrap` object, and it can be printed via `show()` (or retrieved through the `data` attribute). But of course it will be lost if the notebook kernel is terminated. If the processing is very long or important, as a safety measure the `delete=False` parameter to `process()` will cancel file removal.

In [12]:
pk = cc.ProcessWrap( processing_call, 3, "Not deleted!" )
pk.process( delete=False )

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
Done! Msg=Not deleted! Param={}


In [13]:
# See the filename that was used
pk.logname

'/vm/Vagrant/machine-learning-vm/notebook/vmfiles/IPNB/MLCourse/07-DeepLearning/notebook-i9j9tt5i.log'

In [14]:
# The data is there for posterity
# We use the io.open 'newline' argument so that (in Python 3) we avoid newline translation
import io
with io.open(pk.logname,"r",newline='') as f:
    print( f.read() )

  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
Done! Msg=Not deleted! Param={}



In [15]:
# Or perhaps not
import os
os.unlink( pk.logname )

# Low level API
The `ProcessWrap` object functionality is implemented via lower level objects:

* `OutputDest` is a file-like object that substitutes _stdout_ & _stderr_ to perform the data capture
* `ConsoleCapture` provides an object that uses `OutputDest`, and `ConsoleCaptureCtx` a context manager for it
* `ProcessingThread` provides the processing thread that encapsulates its output via `ConsoleCaptureCtx`

These objects can be used directly if more specialised treatment is needed

### ConsoleCaptureCtx


In [16]:
import datetime

with cc.ConsoleCaptureCtx(verbose=False) as ll1:
    for i in range(15):
        print( '{:3}  {}'.format(i+1, datetime.datetime.now()) )

In [17]:
ll1.reprint()

  1  2016-12-27 13:26:39.333098
  2  2016-12-27 13:26:39.333230
  3  2016-12-27 13:26:39.337941
  4  2016-12-27 13:26:39.338006
  5  2016-12-27 13:26:39.338039
  6  2016-12-27 13:26:39.338093
  7  2016-12-27 13:26:39.338119
  8  2016-12-27 13:26:39.338136
  9  2016-12-27 13:26:39.338152
 10  2016-12-27 13:26:39.338179
 11  2016-12-27 13:26:39.338209
 12  2016-12-27 13:26:39.338239
 13  2016-12-27 13:26:39.338269
 14  2016-12-27 13:26:39.338300
 15  2016-12-27 13:26:39.338327


In [18]:
ll1.data

'  1  2016-12-27 13:26:39.333098\n  2  2016-12-27 13:26:39.333230\n  3  2016-12-27 13:26:39.337941\n  4  2016-12-27 13:26:39.338006\n  5  2016-12-27 13:26:39.338039\n  6  2016-12-27 13:26:39.338093\n  7  2016-12-27 13:26:39.338119\n  8  2016-12-27 13:26:39.338136\n  9  2016-12-27 13:26:39.338152\n 10  2016-12-27 13:26:39.338179\n 11  2016-12-27 13:26:39.338209\n 12  2016-12-27 13:26:39.338239\n 13  2016-12-27 13:26:39.338269\n 14  2016-12-27 13:26:39.338300\n 15  2016-12-27 13:26:39.338327\n'

### ConsoleCapture

In [19]:
ll2 = cc.ConsoleCapture()

In [20]:
print( "A")
print( "B" )

A
B


In [21]:
ll2.start()

print( "CDE...\r", end='')
print( "...FGH")
print( "IJK")

ll2.stop();

...FGH
IJK


In [22]:
ll2.reprint().remove();

...FGH
IJK
